In [ ]:
import h5py
import os, urllib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import convolve1d

In [ ]:
def moving_center(X, n, axis=0):
    if n % 2 == 0:
        n += 1
    w = -np.ones(n) / n
    w[n // 2] += 1
    X_ctd = convolve1d(X, w, axis=axis)
    return X_ctd

In [ ]:
def load_sabes_data(filename, bin_width_s=.05, high_pass=True, sqrt=True, thresh=5000,
                    zscore_pos=True):
    # Load MATLAB file
    with h5py.File(filename, "r") as f:
        # Get channel names (e.g. M1 001 or S1 001)
        n_channels = f['chan_names'].shape[1]
        chan_names = []
        for i in range(n_channels):
            chan_names.append(f[f['chan_names'][0, i]][()].tobytes()[::2].decode())
        # Get M1 and S1 indices
        M1_indices = [i for i in range(n_channels) if chan_names[i].split(' ')[0] == 'M1']
        S1_indices = [i for i in range(n_channels) if chan_names[i].split(' ')[0] == 'S1']
        # Get time
        t = f['t'][0, :]
        # Individually process M1 and S1 indices
        result = {}
        for indices in (M1_indices, S1_indices):
            if len(indices) == 0:
                continue
            # Get region (M1 or S1)
            region = chan_names[indices[0]].split(" ")[0]
            # Perform binning
            n_channels = len(indices)
            n_sorted_units = f["spikes"].shape[0] - 1  # The FIRST one is the 'hash' -- ignore!
            d = n_channels * n_sorted_units
            max_t = t[-1]
            n_bins = int(np.floor((max_t - t[0]) / bin_width_s))
            binned_spikes = np.zeros((n_bins, d), dtype=np.int)
            for chan_idx in indices:
                for unit_idx in range(1, n_sorted_units):  # ignore hash!
                    spike_times = f[f["spikes"][unit_idx, chan_idx]][()]
                    if spike_times.shape == (2,):
                        # ignore this case (no data)
                        continue
                    spike_times = spike_times[0, :]
                    # get rid of extraneous t vals
                    spike_times = spike_times[spike_times - t[0] < n_bins * bin_width_s]
                    bin_idx = np.floor((spike_times - t[0]) / bin_width_s).astype(np.int)
                    unique_idxs, counts = np.unique(bin_idx, return_counts=True)
                    # make sure to ignore the hash here...
                    binned_spikes[unique_idxs, chan_idx * n_sorted_units + unit_idx - 1] += counts
            binned_spikes = binned_spikes[:, binned_spikes.sum(axis=0) > thresh]
            if sqrt:
                binned_spikes = np.sqrt(binned_spikes)
            if high_pass:
                binned_spikes = moving_center(binned_spikes, n=600)
            result[region] = binned_spikes
        # Get cursor position
        cursor_pos = f["cursor_pos"][:].T
        # Line up the binned spikes with the cursor data
        t_mid_bin = np.arange(len(binned_spikes)) * bin_width_s + bin_width_s / 2
        cursor_pos_interp = interp1d(t - t[0], cursor_pos, axis=0)
        cursor_interp = cursor_pos_interp(t_mid_bin)
        if zscore_pos:
            cursor_interp -= cursor_interp.mean(axis=0, keepdims=True)
            cursor_interp /= cursor_interp.std(axis=0, keepdims=True)
        result["cursor"] = cursor_interp
        # Get target position
        target_pos = f["target_pos"][:].T
        target_pos_interp = interp1d(t - t[0], target_pos, axis=0)
        target_interp = target_pos_interp(t_mid_bin)
        if zscore_pos:
            target_interp -= target_interp.mean(axis=0, keepdims=True)
            target_interp /= target_interp.std(axis=0, keepdims=True)
        result["target"] = target_interp
        return result

In [ ]:
fname = 'indy_20160627_01.mat'

In [ ]:
data = load_sabes_data(fname, bin_width_s=.05)


In [ ]:
Y = data['cursor']


In [ ]:
len(Y)

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(*Y[:1200].T, c='k')
plt.xlabel('cursor x')
plt.ylabel('cursor y')

In [ ]:
def calculate_velocity(cursor_interp):
   # 计算速度矩阵
   velocity = np.gradient(cursor_interp, axis=0)
   return velocity

velocity = calculate_velocity(Y)


In [ ]:
velocity

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(*velocity[:1200].T, c='k')
plt.xlabel('velocity x')
plt.ylabel('velocity y')

In [ ]:
acc = calculate_velocity(velocity)
plt.figure(figsize=(5, 5))
plt.plot(*acc[:1200].T, c='k')
plt.xlabel('acc x')
plt.ylabel('acc y')

In [ ]:
def generate_lidar(distance, standard_deviation):
	ran = np.random.normal(0, standard_deviation, (len(distance),2))
	return distance + ran

In [ ]:
T=data['target']

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(*T[:1200].T, c='k')
plt.xlabel('cursor x')
plt.ylabel('cursor y')

In [ ]:
tvelocity = calculate_velocity(T)
plt.figure(figsize=(5, 5))
plt.plot(*tvelocity[:1200].T, c='k')
plt.xlabel('velocity x')
plt.ylabel('velocity y')

In [ ]:
tacc = calculate_velocity(tvelocity)
plt.figure(figsize=(5, 5))
plt.plot(*tacc[:1200].T, c='k')
plt.xlabel('cursor x')
plt.ylabel('cursor y')

In [ ]:
import matrix as m
import math
initial_distance = 0
initial_velocity = 0
x_initial = m.Matrix([[initial_distance], [initial_velocity * 1e-3 / (60 * 60)]])
P_initial = m.Matrix([[5, 0],[0, 5]])
standard_deviation = 0.1

In [ ]:
def F_matrix(delta_t):
    return m.Matrix([[1, delta_t], [0, 1]])

def Q_matrix(delta_t, variance):
    t4 = math.pow(delta_t, 4)
    t3 = math.pow(delta_t, 3)
    t2 = math.pow(delta_t, 2)
    
    return variance * m.Matrix([[(1/4)*t4, (1/2)*t3], [(1/2)*t3, t2]])

In [ ]:
lidar_variance = standard_deviation**2
H = m.Matrix([[1, 0]])
R = m.Matrix([[lidar_variance]])
I = m.identity(2)

In [ ]:
# Kalman Filter Implementation

x = x_initial
P = P_initial

x_result = []
time_result = []
v_result = []
acceleration_variance = 20

for i in range(len(Y) - 1):
        
    # calculate time that has passed between lidar measurements
    delta_t = 0.05

    # Prediction Step - estimates how far the object traveled during the time interval
    F = F_matrix(delta_t)
    Q = Q_matrix(delta_t, acceleration_variance)
    
    x_prime = F * x
    P_prime = F * P * F.T() + Q
    
    # Measurement Update Step
    y = m.Matrix([[T[i + 1]]]) - H * x_prime
    S = H * P_prime * H.T() + R
    K = P_prime * H.T() * S.inverse()
    x = x_prime + K * y
    P = (I - K * H) * P_prime

    # Store distance and velocity belief and current time
    x_result.append(x[0][0])

In [ ]:

result = np.array([list(x) for x in x_result])
result

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(*result[:1200].T, c='k')
plt.xlabel('cursor x')
plt.ylabel('cursor y')

In [ ]:
def get_R2(y_test,y_test_pred):

    """
    Function to get R2

    Parameters
    ----------
    y_test - the true outputs (a matrix of size number of examples x number of outputs)
    y_test_pred - the predicted outputs (a matrix of size number of examples x number of outputs)

    Returns
    -------
    R2_array: An array of R2s for each output
    """

    R2_list=[] #Initialize a list that will contain the R2s for all the outputs
    for i in range(y_test.shape[1]): #Loop through outputs
        #Compute R2 for each output
        y_mean=np.mean(y_test[:,i])
        R2=1-np.sum((y_test_pred[:,i]-y_test[:,i])**2)/np.sum((y_test[:,i]-y_mean)**2)
        R2_list.append(R2) #Append R2 of this output to the list
    R2_array=np.array(R2_list)
    return R2_array #Return an array of R2s

In [ ]:
len(result)


In [ ]:
len(T)

In [ ]:
r2 = get_R2(result,T[1:])
r2